# <center>Network Science</center>

## <center>Home Assignment #3: Centralities and Assortativity</center>

### <center>Student: Maxim Ushakov</center>

#### <hr /> General Information

**Due Date:** 13.03.2016 23:59 <br \>
**Late submission policy:** -0.2 points per day <br \>


Please send your reports to <mailto:network.hse.2016@gmail.com> with message subject of the following structure:<br \> **[HSE Networks 2015] *{LastName}* *{First Name}* HA*{Number}***

Support your computations with figures and comments. <br \>
If you are using IPython Notebook you may use this file as a starting point of your report.<br \>
<br \>
<hr \>

## Problems

### Task 1

Compute degree centrality, Pagerank and  HubAuthorities scores for the [flickr](https://snap.stanford.edu/data/web-flickr.html) network. 

Data contains sparse matrix A and list of user names.
This is a “denser” part of the Flickr photo sharing site friendship graph from 2006. Edge direction corresponds to friendship requests (following). Some of the links are reciprocal,others not.  

Provide top 50 names in each ranking, compare results

## Solution
We start with Pagerank score, then we calculate Hub and Authorities cores, then we compute incoming and outcoming degree centralities, and, finally, we compare obtained results.
Let us download the given data.

In [1]:
import scipy.io
import numpy as np
import scipy.sparse
import scipy.sparse.linalg

data = scipy.io.loadmat('flickr.mat')
A = data['A']
names = data['names']
num_vertices = A.shape[0]

Check out that all vertices have outcoming edges:

In [2]:
print(np.sum(np.array(A.sum(1)>0)))

15724


As all vertices have outcoming edges (there is no absorbing node in our graph), than this graph satisfies Perron-Frobenius theorem (without any modifications). We use the formula:
$$p=\alpha \cdot (D^{-1}\cdot A)^T\cdot p + (1-\alpha)\cdot e/n$$,
to compute Pagerank score.
By setting regularization coefficient equal to 0.999, we obtain matrices $B=\alpha \cdot (D^{-1}\cdot A)^T$ and $C=(1-\alpha)\cdot e/n$. We will need them to compute Pagerank score $p$.

In [3]:
alpha = 0.999
P = scipy.sparse.lil_matrix(A)
D_diag = A.sum(1)
for i in range(num_vertices):
    P[i,] /= D_diag[i]

In [6]:
pagerank_score = np.ones(num_vertices)/num_vertices
B = alpha*P.transpose()
C = (1-alpha)*np.ones(num_vertices)/num_vertices

I've tried to find $p$ by solving system $(I-B)\cdot p=C$, but, unfortunately, jupyter's kernel has died during computations, so I've decided to solve the system iteratively. To ensure uniform convergence of $p$ to real Pagerank score, we set up maximal error value $eps \approx 1/30000$. Through 10 iterations we obtain Pagerank score with given accuracy. 

In [7]:
pr_pagerank_score = np.zeros(num_vertices)/num_vertices
eps = 1.0 / (2*num_vertices)

num_try = 0
while np.max(np.abs(pagerank_score - pr_pagerank_score)) > eps:
    pr_pagerank_score = pagerank_score
    pagerank_score = B*pagerank_score + C
    
    num_try += 1
    print(num_try, np.max(np.abs(pagerank_score - pr_pagerank_score)))

1 0.00216565412268
2 0.000380384142822
3 0.000137013658883
4 6.97535184881e-05
5 5.05494776548e-05
6 4.25222718043e-05
7 3.87872010842e-05
8 3.55731338804e-05
9 3.31503206455e-05
10 3.08481563947e-05


Here we have names of 50 users with highest Pagerank score (presented in descending order). 

In [62]:
names = [name.strip(' ') for name in names]
pagerank_score_names = [(names[i], pagerank_score[i]) for i in range(num_vertices)]
sorted_names_pagerank = sorted(pagerank_score_names, key=lambda obj: obj[1], reverse=True)
for name in sorted_names_pagerank[:50]:
    print(name[0])

awfulsara
drp
BombDog
antimethod
*Ivan*
DrJoanne
Simon Pais
cymagen
deborah lattimore
:Nikola
slowernet
jkottke
notraces
artofgold
MaD GiÂ®Lâ¢â¢
Pandarine
lorrainemd
romanlily
*starlet*
fraying
anildash
Mylens
gu@va
underbunny
Mareen Fischinger
maximolly
Loobylu
* HoNe$t *
Joi
pbowers
Tom Coates
gruntzooki
pixietart
bernardo.borghetti
chromogenic
Gayla
Departure Lounge
Marcelo  Montecino
overshadowed
Esther_G
jakedobkin
aleyna
990000
CherryVega
hot_luscious
rion
macwagen
chacabuco
pablokorona
Agridulce


To compute Hub and Authority scores we use formulas:
$$(A^T A) \cdot a = \lambda \cdot a,$$
$$(A A^T) \cdot h = \lambda \cdot h,$$
where $\lambda$ is a maximum eigenvalue.

In [60]:
A_temp = A*A.transpose()
l, h_vector = scipy.sparse.linalg.eigs(A_temp,k=1)
A_temp = A.transpose()*A
l2, a_vector = scipy.sparse.linalg.eigs(A_temp,k=1)

Check out that maximum eigenvalues are the same.

In [61]:
print(l, l2)

[ 6153.46853672+0.j] [ 6153.46853672+0.j]


Here we have names of 50 users with highest Hub score (presented in descending order). 

In [63]:
hub_score_names = [(names[i], h_vector[i]) for i in range(num_vertices)]
sorted_names_hub = sorted(hub_score_names, key=lambda obj: obj[1], reverse=True)
for name in sorted_names_hub[:50]:
    print(name[0])

florecita120
agrypnia
Lutrus
Fellowship of the Rich
HikingVal
francesgatz
nomad7674
Princess Lou
Karl Kanal
Jacki Gordon
Bonnie Christiana (Quibell)
Belinda 05
Miss CC
jcruelty
TheRealOdie
hkboyee
snkutty
caffcaff
heven2mrgatroid
Sylla
sulaco_rm
misseill
Termo
qbqrat
louloubelle2
arlo
MissNicky
Gimelo
hotdiggidydog
slightlydope
Kir0n
_Yodi
JimisonPix
wasabi1809
DawgPound
Melanie Maynor
Andrew Mac
3a|M|atko0m 7ub 7ub
waration
speedyplatinum
sazzz
bobben
twcheong
MrKev
AraBiC
Zars
eve_
Nicki & Jase
naleslie
CRM


Here we have names of 50 users with highest Authority score (presented in descending order). 

In [64]:
authority_score_names = [(names[i], a_vector[i]) for i in range(num_vertices)]
sorted_names_authority = sorted(authority_score_names, key=lambda obj: obj[1], reverse=True)
for name in sorted_names_authority[:50]:
    print(name[0])

francesgatz
HikingVal
florecita120
caffcaff
Bonnie Christiana (Quibell)
Jacki Gordon
Belinda 05
Gimelo
Miss CC
babepy
agrypnia
snkutty
Wati
louloubelle2
eingy
sazzz
speedyplatinum
Awlad-Alhowa
twcheong
Extra Girl
james and danielle
MrKev
DawgPound
hotdiggidydog
heven2mrgatroid
Miss_Gossip
ÒÃ¤ÅÄ·ÅÅÅÃ¤
Lady Birchwood
miss pepsi
ShÃ´jo
_Yodi
MissNicky
Justin B
qbqrat
Loefty
TheRealOdie
gigiloo
TruckstopC
AraBiC
I'mMessy
B...
misseill
mohammed3
slightlydope
KaiFy_EmaRaTy
Melanie Maynor
waration
FraukeFoto
*&#@UniQuE@#&*
(Â¯`Â·._.Â·[Mr.MaZyo0oN]Â·._.Â·Â´Â¯)


Finally, we calculate indegree and outdegree centrality (simply as number of incoming/outcoming vertices).

In [65]:
outdegree_centrality = A.sum(1)
indegree_centrality = A.sum(0)

Here we have names of 50 users with highest outdegree centrality (presented in descending order). 

In [66]:
outdegree_score_names = [(names[i], outdegree_centrality[i]) for i in range(num_vertices)]
sorted_names_outdegree = sorted(outdegree_score_names, key=lambda obj: obj[1], reverse=True)
for name in sorted_names_outdegree[:50]:
    print(name[0])

anildash
tozzer
AtiRanA
pixietart
jakedobkin
Buntekuh
brainware3000
Jakes_World
maximolly
Andreia Lopes
elvis1967
inthegan
990000
: Esther
dmoore
kathryn
Charles Machado
spanier
drp
What What
rivello
mehmetkale
blackbeltjones
dianeham
Lucilia
meadows
aracay
pro keds
a nameless yeast
cooks
nickf
virgu
tantek
Zsaj
Photographer Dre
jayallen
cymagen
gardengal
me4ta
mypapercrane
judith
Airchild
ribena
roamin
mdintoronto
lucycat
killer robot
chomp
Waldgeister
ghostbones


Here we have names of 50 users with highest indegree centrality (presented in descending order). 

In [68]:
indegree_score_names = [(names[i], indegree_centrality[0,i]) for i in range(num_vertices)]
sorted_names_indegree = sorted(indegree_score_names, key=lambda obj: obj[1], reverse=True)
for name in sorted_names_indegree[:50]:
    print(name[0])

awfulsara
drp
*Ivan*
antimethod
DrJoanne
BombDog
Simon Pais
deborah lattimore
MaD GiÂ®Lâ¢â¢
notraces
:Nikola
cymagen
aleyna
lorrainemd
artofgold
*starlet*
romanlily
Pandarine
jkottke
hot_luscious
Mareen Fischinger
Mylens
slowernet
gu@va
bernardo.borghetti
CherryVega
pbowers
underbunny
Loobylu
Merina
* HoNe$t *
.lush
fraying
pixietart
naftalina007
fd
aquanerds
Joi
Esther_G
carf
Marcelo  Montecino
!!uAe prince!!
Sexy Swedish Babe
callipygian
Agridulce
Least Wanted
Gayla
anildash
tecgirl
reddirtrose


To compare results, we compute Spearman rank correlation coefficient between every pair of scores. 

In [69]:
ranking = {}
for name in names:
    ranking[name] = {'page': 0, 'hub': 0, 'auth': 0, 'indeg': 0, 'outdeg': 0}
for ind in range(num_vertices):
    ranking[sorted_names_pagerank[ind][0]]['page']=ind
    ranking[sorted_names_hub[ind][0]]['hub']=ind
    ranking[sorted_names_authority[ind][0]]['auth']=ind
    ranking[sorted_names_indegree[ind][0]]['indeg']=ind
    ranking[sorted_names_outdegree[ind][0]]['outdeg']=ind

In [70]:
corr = {'page-hub': 0, 'page-auth': 0, 'page-indeg': 0, 'page-outdeg': 0,\
        'hub-auth': 0, 'hub-indeg': 0, 'hub-outdeg': 0,\
        'auth-indeg': 0, 'auth-outdeg': 0,\
        'indeg-outdeg': 0}
for name, ranks in ranking.items():
    corr['page-hub'] += ranks['page']*ranks['hub']
    corr['page-auth'] += ranks['page']*ranks['auth']
    corr['page-indeg'] += ranks['page']*ranks['indeg']
    corr['page-outdeg'] += ranks['page']*ranks['outdeg']
    corr['hub-auth'] += ranks['hub']*ranks['auth']
    corr['hub-indeg'] += ranks['hub']*ranks['indeg']
    corr['hub-outdeg'] += ranks['hub']*ranks['outdeg']
    corr['auth-indeg'] += ranks['auth']*ranks['indeg']
    corr['auth-outdeg'] += ranks['auth']*ranks['outdeg']
    corr['indeg-outdeg'] += ranks['indeg']*ranks['outdeg']

middle_corr = num_vertices*(num_vertices-1)*(num_vertices-1)/4
half_corr = num_vertices*(num_vertices+1)*(num_vertices-1)/12

corr['page-hub'] = (corr['page-hub']-middle_corr)/half_corr
corr['page-auth'] = (corr['page-auth']-middle_corr)/half_corr
corr['page-indeg'] = (corr['page-indeg']-middle_corr)/half_corr
corr['page-outdeg'] = (corr['page-outdeg']-middle_corr)/half_corr
corr['hub-auth'] = (corr['hub-auth']-middle_corr)/half_corr
corr['hub-indeg'] = (corr['hub-indeg']-middle_corr)/half_corr
corr['hub-outdeg'] = (corr['hub-outdeg']-middle_corr)/half_corr
corr['auth-indeg'] = (corr['auth-indeg']-middle_corr)/half_corr
corr['auth-outdeg'] = (corr['auth-outdeg']-middle_corr)/half_corr
corr['indeg-outdeg'] = (corr['indeg-outdeg']-middle_corr)/half_corr

print(corr['page-hub'])
print(corr['page-auth'])
print(corr['page-indeg'])
print(corr['page-outdeg'])
print(corr['hub-auth'])
print(corr['hub-indeg'])
print(corr['hub-outdeg'])
print(corr['auth-indeg'])
print(corr['auth-outdeg'])
print(corr['indeg-outdeg'])

-0.5139048999754795
-0.7532878626704752
0.9466561676018755
0.757843223375695
0.811425161790339
-0.5346320999834844
-0.7338858865326491
-0.7922200536475691
-0.705067233892226
0.8144566272754413


As we can see, there is high correlation between Pagerank score and indegree/outdegree centralities. It seems reasonable, as Pagerank score shows probability for node to be visited during random walk, and if node has many neighbors, then its probability to be visited is high. Also we can see that there is high relation between Hub and Authority scores. It also seems believable, as most popular nodes have high probability to be referenced, and vise versa, popular nodes are such nodes, that reference to other popular nodes. Finally, we can see negative correlation between Pagerank, indegree/outdegree scores and Hub and Authority scores.

### <hr /> Task 2

Here are the [Facebook friendship graphs](https://snap.stanford.edu/data/egonets-Facebook.html) from several US universities from 2005 (one year after fb launch).

Data contains a A matrix (sparse) and a "local_info" variable, one row per node: 
a student/faculty status flag, gender, major, second major/minor (if applicable), dorm/house, year, and high school. 
Missing data is coded 0.

Compute node degree assortativity (mixining by node degree) and assortativity coefficient (modularity) for gender, major, dormitory, year, high school for all universities and compare the results.

## Solution
Here we calculate degree assortativity and assortativity coefficients for every university's facebook network.

Let's start with Berkeley university. First, we download the data.

In [76]:
data2 = scipy.io.loadmat('Berkeley13.mat')
A2 = data2['A']
local_info = data2['local_info']
num_vertices2, num_attributes = local_info.shape

Then we calculate degree assortativity by using the formula: 
$$r = \frac{Se\cdot S1-S2\cdot S2}{S3\cdot S1-S2\cdot S2},$$
where $S1 = 2m$, $S2 = \sum_i k_i^2$, $S3 = \sum_i k_i^3$, $Se = \sum_{i,j} k_i\cdot A_{i,j}\cdot k_j$

In [90]:
k = np.array(A2.sum(1))
S1 = k.sum()
S2 = (k*k).sum()
S3 = (k*k*k).sum()
Se = np.matrix(k).transpose()*A2*np.matrix(k)
r = (Se*S1-S2*S2)/(S3*S1-S2*S2)
print(r)

[[ 0.01031346]]


Then we calculate assortativity coefficients using the formula:
$$\frac{Q}{Q_{max}}=\frac{\sum_{i,j}(A_{ij}-\frac{k_i k_j}{2m})\delta (c_i,c_j)}{2m-\sum_{i,j}(\frac{k_i k_j}{2m})\delta (c_i,c_j)}$$

In [102]:
attrib_check = [1,2,4,5,6]
modularity = {1:0,2:0,4:0,5:0,6:0}
for attr in attrib_check:
    attr_classes = {}
    #print(attr)
    for j in range(num_vertices2):
        try:
            attr_classes[local_info[j, attr]].append(j)
        except:
            attr_classes[local_info[j, attr]] = [j]
    #print(len(attr_classes))
    max_modularity = S1
    for attr_v, ind_list in attr_classes.items():
        k_class = k[ind_list]
        A_class = A2[ind_list,ind_list]
        temp_sum = sum(k_class)**2/S1
        max_modularity -= temp_sum
        #print(temp_sum, max_modularity)
        modularity[attr] += A_class.sum()-temp_sum
    modularity[attr] /= max_modularity
    #print(modularity[attr], max_modularity)
    
print(modularity)      

{1: array([-0.80652993]), 2: array([-0.04454065]), 4: array([-0.61309089]), 5: array([-0.20304273]), 6: array([-0.01515052])}


The same thing we do for other universities.

In [103]:
data2 = scipy.io.loadmat('Caltech36.mat')
A2 = data2['A']
local_info = data2['local_info']
num_vertices2, num_attributes = local_info.shape
k = np.array(A2.sum(1))
S1 = k.sum()
S2 = (k*k).sum()
S3 = (k*k*k).sum()
Se = np.matrix(k).transpose()*A2*np.matrix(k)
r = (Se*S1-S2*S2)/(S3*S1-S2*S2)
print(r)
attrib_check = [1,2,4,5,6]
modularity = {1:0,2:0,4:0,5:0,6:0}
for attr in attrib_check:
    attr_classes = {}
    #print(attr)
    for j in range(num_vertices2):
        try:
            attr_classes[local_info[j, attr]].append(j)
        except:
            attr_classes[local_info[j, attr]] = [j]
    #print(len(attr_classes))
    max_modularity = S1
    for attr_v, ind_list in attr_classes.items():
        k_class = k[ind_list]
        A_class = A2[ind_list,ind_list]
        temp_sum = sum(k_class)**2/S1
        max_modularity -= temp_sum
        #print(temp_sum, max_modularity)
        modularity[attr] += A_class.sum()-temp_sum
    modularity[attr] /= max_modularity
    #print(modularity[attr], max_modularity)
    
print(modularity)  

[[-0.06527295]]
{1: array([-0.98182203]), 2: array([-0.08223703]), 4: array([-0.13632138]), 5: array([-0.27276695]), 6: array([-0.00889961])}


In [104]:
data2 = scipy.io.loadmat('Harvard1.mat')
A2 = data2['A']
local_info = data2['local_info']
num_vertices2, num_attributes = local_info.shape
k = np.array(A2.sum(1))
S1 = k.sum()
S2 = (k*k).sum()
S3 = (k*k*k).sum()
Se = np.matrix(k).transpose()*A2*np.matrix(k)
r = (Se*S1-S2*S2)/(S3*S1-S2*S2)
print(r)
attrib_check = [1,2,4,5,6]
modularity = {1:0,2:0,4:0,5:0,6:0}
for attr in attrib_check:
    attr_classes = {}
    #print(attr)
    for j in range(num_vertices2):
        try:
            attr_classes[local_info[j, attr]].append(j)
        except:
            attr_classes[local_info[j, attr]] = [j]
    #print(len(attr_classes))
    max_modularity = S1
    for attr_v, ind_list in attr_classes.items():
        k_class = k[ind_list]
        A_class = A2[ind_list,ind_list]
        temp_sum = sum(k_class)**2/S1
        max_modularity -= temp_sum
        #print(temp_sum, max_modularity)
        modularity[attr] += A_class.sum()-temp_sum
    modularity[attr] /= max_modularity
    #print(modularity[attr], max_modularity)
    
print(modularity)  

[[ 0.14505489]]
{1: array([-0.72935764]), 2: array([-0.06568939]), 4: array([-0.07259287]), 5: array([-0.16229399]), 6: array([-0.03064139])}


In [105]:
data2 = scipy.io.loadmat('Oklahoma97.mat')
A2 = data2['A']
local_info = data2['local_info']
num_vertices2, num_attributes = local_info.shape
k = np.array(A2.sum(1))
S1 = k.sum()
S2 = (k*k).sum()
S3 = (k*k*k).sum()
Se = np.matrix(k).transpose()*A2*np.matrix(k)
r = (Se*S1-S2*S2)/(S3*S1-S2*S2)
print(r)
attrib_check = [1,2,4,5,6]
modularity = {1:0,2:0,4:0,5:0,6:0}
for attr in attrib_check:
    attr_classes = {}
    #print(attr)
    for j in range(num_vertices2):
        try:
            attr_classes[local_info[j, attr]].append(j)
        except:
            attr_classes[local_info[j, attr]] = [j]
    #print(len(attr_classes))
    max_modularity = S1
    for attr_v, ind_list in attr_classes.items():
        k_class = k[ind_list]
        A_class = A2[ind_list,ind_list]
        temp_sum = sum(k_class)**2/S1
        max_modularity -= temp_sum
        #print(temp_sum, max_modularity)
        modularity[attr] += A_class.sum()-temp_sum
    modularity[attr] /= max_modularity
    #print(modularity[attr], max_modularity)
    
print(modularity)  

[[ 0.07367326]]
{1: array([-0.86190096]), 2: array([-0.02283702]), 4: array([-0.25421464]), 5: array([-0.20131707]), 6: array([-0.0154671])}


In [106]:
data2 = scipy.io.loadmat('Princeton12.mat')
A2 = data2['A']
local_info = data2['local_info']
num_vertices2, num_attributes = local_info.shape
k = np.array(A2.sum(1))
S1 = k.sum()
S2 = (k*k).sum()
S3 = (k*k*k).sum()
Se = np.matrix(k).transpose()*A2*np.matrix(k)
r = (Se*S1-S2*S2)/(S3*S1-S2*S2)
print(r)
attrib_check = [1,2,4,5,6]
modularity = {1:0,2:0,4:0,5:0,6:0}
for attr in attrib_check:
    attr_classes = {}
    #print(attr)
    for j in range(num_vertices2):
        try:
            attr_classes[local_info[j, attr]].append(j)
        except:
            attr_classes[local_info[j, attr]] = [j]
    #print(len(attr_classes))
    max_modularity = S1
    for attr_v, ind_list in attr_classes.items():
        k_class = k[ind_list]
        A_class = A2[ind_list,ind_list]
        temp_sum = sum(k_class)**2/S1
        max_modularity -= temp_sum
        #print(temp_sum, max_modularity)
        modularity[attr] += A_class.sum()-temp_sum
    modularity[attr] /= max_modularity
    #print(modularity[attr], max_modularity)
    
print(modularity)  

[[ 0.09109212]]
{1: array([-0.76646806]), 2: array([-0.07374385]), 4: array([-0.11531597]), 5: array([-0.20038218]), 6: array([-0.0211872])}


In [107]:
data2 = scipy.io.loadmat('Texas80.mat')
A2 = data2['A']
local_info = data2['local_info']
num_vertices2, num_attributes = local_info.shape
k = np.array(A2.sum(1))
S1 = k.sum()
S2 = (k*k).sum()
S3 = (k*k*k).sum()
Se = np.matrix(k).transpose()*A2*np.matrix(k)
r = (Se*S1-S2*S2)/(S3*S1-S2*S2)
print(r)
attrib_check = [1,2,4,5,6]
modularity = {1:0,2:0,4:0,5:0,6:0}
for attr in attrib_check:
    attr_classes = {}
    #print(attr)
    for j in range(num_vertices2):
        try:
            attr_classes[local_info[j, attr]].append(j)
        except:
            attr_classes[local_info[j, attr]] = [j]
    #print(len(attr_classes))
    max_modularity = S1
    for attr_v, ind_list in attr_classes.items():
        k_class = k[ind_list]
        A_class = A2[ind_list,ind_list]
        temp_sum = sum(k_class)**2/S1
        max_modularity -= temp_sum
        #print(temp_sum, max_modularity)
        modularity[attr] += A_class.sum()-temp_sum
    modularity[attr] /= max_modularity
    #print(modularity[attr], max_modularity)
    
print(modularity)  

[[ 0.16389241]]
{1: array([-0.878421]), 2: array([-0.02763613]), 4: array([-0.35215334]), 5: array([-0.19798115]), 6: array([-0.00792501])}


To conclude, node degree assortativity is usually approximately equal to 0, so the same number of friends is not a reason for being related. Also we can see that all modularities are below zero, which means that in our cases we have Disassortative mixing (users with different attributes are tend to be related), especially by gender.